### https://www.kaggle.com/competitions/drawing-with-llms

In [35]:
import kagglehub
import pandas as pd

train_path = kagglehub.competition_download('drawing-with-llms', 'train.csv')
train = pd.read_csv(train_path)

train.head()

,id,description
0,04c411,a starlit night over snow-covered peaks
1,215136,black and white checkered pants
2,3e2bc6,crimson rectangles forming a chaotic grid
3,61d7a8,burgundy corduroy pants with patch pockets and...
4,6f2ca7,orange corduroy overalls


In [36]:
import openai
import pandas as pd
import time
import os
from dotenv import load_dotenv
load_dotenv()
openai_api_key = os.getenv("OPENAI_API_KEY")

In [37]:
# Set up OpenAI client with your API key
client = openai.OpenAI(api_key=openai_api_key)

# Function to get summary from OpenAI API
def get_summary(text):
    instruction = f"""Generate SVG code to visually represent the following text description, while respecting the given constraints.
    
    <constraints>
    * **Allowed Elements:** `svg`, `path`, `circle`, `rect`, `ellipse`, `line`, `polyline`, `polygon`, `g`, `linearGradient`, `radialGradient`, `stop`, `defs`
    * **Allowed Attributes:** `viewBox`, `width`, `height`, `fill`, `stroke`, `stroke-width`, `d`, `cx`, `cy`, `r`, `x`, `y`, `rx`, `ry`, `x1`, `y1`, `x2`, `y2`, `points`, `transform`, `opacity`
    </constraints>

    <example>
    <description>"A red circle with a blue square inside"</description>
    ```svg
    <svg viewBox="0 0 256 256" width="256" height="256">
      <circle cx="50" cy="50" r="40" fill="red"/>
      <rect x="30" y="30" width="40" height="40" fill="blue"/>
    </svg>
    ```
    </example>

    Please ensure that the generated SVG code is well-formed, valid, and strictly adheres to these constraints.
    Focus on a clear and concise representation of the input description within the given limitations. 
    Always provide the complete SVG code with nothing omitted. Never use an ellipsis.

    <description>"{text}"</description>
    """

    try:
        response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=[
                {"role": "system", "content": "Generate an SVG code with the given instruction."},
                {"role": "user", "content": instruction}
            ],
            temperature=0.5,
            max_tokens=1024
        )
        return response.choices[0].message.content.strip()
    
    except Exception as e:
        print(f"Error: {e}")
        return None


In [38]:
from tqdm.auto import tqdm

tqdm.pandas()  # Enable progress bar for pandas
train["svg_code"] = train["description"].progress_apply(get_summary)

100%|███████████████████████████████████████████| 15/15 [00:34<00:00,  2.28s/it]


In [39]:
svg_code

'<svg viewBox="0 0 100 100" width="100" height="100" xmlns="http://www.w3.org/2000/svg">\n  <rect x="0" y="0" width="50" height="50" fill="black"/>\n  <rect x="50" y="0" width="50" height="50" fill="white"/>\n  <rect x="0" y="50" width="50" height="50" fill="white"/>\n  <rect x="50" y="50" width="50" height="50" fill="black"/>\n</svg>'

In [40]:
# We can play with our Model and render its SVG output (don't export!)
from IPython.display import SVG, display
for svg in train["svg_code"]:
    display(SVG(svg))

ExpatError: not well-formed (invalid token): line 1, column 0

In [ ]:
train